In [ ]:
import logging
import matplotlib.pyplot as plt
import numpy as np

# Import the main class
from main import MainOptimizer
logger = logging.getLogger()
logger.setLevel(logging.INFO)

print("Setup complete. Ready to initialize the optimizer.")


In [ ]:
try:
    optimizer = MainOptimizer()
    
    # Start the optimization process
    optimizer.run()
    
except SystemExit as e:
    print(f"Optimizer stopped with a critical configuration error: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
    # Use the logging traceback for more detail
    logging.exception("Detailed traceback:")

print("\n--- Optimization Finished ---")

### The following script parses the ga_checkpoint.json file to find the optimal fuel enrichment pattern from the completed optimization run. It identifies the best solution by locating the cycle with the highest stored fitness score and then extracts that cycle's corresponding enrichment values, k-effective, and power peaking factor.

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np

# --- Define Parameters for External Fitness Calculation ---
TARGET_KEFF = 1.12437               # From simulation config
KEFF_PENALTY_FACTOR = 20            # From GA tuning config
PPF_WEIGHT = 0.6                    # User-defined weight
KEFF_WEIGHT = 0.4                   # User-defined weight

In [ ]:
def calculate_external_fitness(ppf: float, keff: float) -> float:
    """
    Calculates a stable, external fitness score with a continuous penalty function.
    """
    keff_diff = abs(keff - TARGET_KEFF)
        
    # Invert PPF so that a lower PPF results in a higher score.
    ppf_score = 1.0 / ppf if ppf > 0 else 0

    # Keff score is calculated using a continuous exponential function.
    # This ensures that any improvement in keff results in a better score
    keff_score = np.exp(-KEFF_PENALTY_FACTOR * keff_diff)

    return PPF_WEIGHT * ppf_score + KEFF_WEIGHT * keff_score

# --- Load the JSON checkpoint file ---
try:
    # Update the path to where your new checkpoint file is saved
    with open('data/ga_checkpoint.json', 'r') as file:
        data = json.load(file)
except FileNotFoundError:
    print("Error: 'data/ga_checkpoint.json' not found. Please ensure the path is correct.")
    exit(1)
except json.JSONDecodeError:
    print("Error: Could not decode JSON. The checkpoint file might be corrupted or empty.")
    exit(1)
except Exception as e:
    print(f"Unexpected error while loading JSON file: {e}")
    exit(1)

# --- Extract Data from the 'history' List ---
history = data.get('history')
if not history:
    print("Error: 'history' key not found or is empty in the checkpoint file.")
    exit(1)

# --- Find the best cycle by recalculating fitness externally ---
best_external_fitness = -np.inf
best_cycle_data_external = None

for cycle_record in history:
    # Extract the true keff and ppf for the current cycle
    current_keff = cycle_record.get('keff')
    current_ppf = cycle_record.get('ppf')

    # Skip any records that might be missing data
    if current_keff is None or current_ppf is None:
        continue

    # Calculate the new fitness score using the external function
    external_fitness = calculate_external_fitness(ppf=current_ppf, keff=current_keff)

    # If this is the best fitness found so far, store the entire record
    if external_fitness > best_external_fitness:
        best_external_fitness = external_fitness
        best_cycle_data_external = cycle_record
        # Add the newly calculated fitness to the record for easy access later
        best_cycle_data_external['external_fitness'] = external_fitness

# --- Display the results ---
if best_cycle_data_external:
    # Extract details from the best cycle found
    best_cycle_num = best_cycle_data_external.get('cycle')
    best_keff = best_cycle_data_external.get('keff')
    best_ppf = best_cycle_data_external.get('ppf')
    best_individual = best_cycle_data_external.get('individual')
    # Use the newly calculated "external_fitness"
    recalculated_best_fitness = best_cycle_data_external.get('external_fitness')

    average_enrichment = None
    if best_individual:
        average_enrichment = np.mean(best_individual)

    print("--- Analysis of ga_checkpoint.json (Using External Fitness Calculation) ---")
    print(f"The cycle with the highest *recalculated* external fitness is: {best_cycle_num}")
    print(f"The highest recalculated fitness is: {recalculated_best_fitness:.6f}")
    print(f"The corresponding Keff for this cycle is: {best_keff:.6f}")
    print(f"The corresponding PPF for this cycle is: {best_ppf:.6f}")
    print(f"The corresponding enrichment values are: {best_individual}")
    if average_enrichment is not None:
        print(f"The average enrichment for this individual is: {average_enrichment:.6f}")
else:
    print("Could not determine the best individual. The history might be empty or missing keff/ppf data.")

### The following script loads historical data from ga_checkpoint file, extracting the fitness scores and the percentage errors for both PPF and k-effective from each cycle. It then uses matplotlib to generate and save three separate plots that visualize the trends of these key performance metrics over the entire optimization run.

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np

# =======================================================
# === 1. Parameters & External Fitness Calculation    ===
# =======================================================
# Change it based on your requirements.
TARGET_KEFF = 1.12437           # From simulation config
KEFF_PENALTY_FACTOR = 20        # From GA tuning config
PPF_WEIGHT = 0.6                # User-defined weight for stable analysis
KEFF_WEIGHT = 0.4               # User-defined weight for stable analysis

def calculate_external_fitness(ppf: float, keff: float) -> float:
    """
    Calculates a stable, external fitness score with a continuous penalty function.
    """
    keff_diff = abs(keff - TARGET_KEFF)
    
    # Invert PPF so that a lower PPF results in a higher score.
    ppf_score = 1.0 / ppf if ppf > 0 else 0

    # Keff score is calculated using a continuous exponential function.
    # This ensures that any improvement in keff results in a better score.
    keff_score = np.exp(-KEFF_PENALTY_FACTOR * keff_diff)

    return PPF_WEIGHT * ppf_score + KEFF_WEIGHT * keff_score

# =======================================================
# === 2. Load and Extract Data                        ===
# =======================================================
try:
    # Update the path to where your checkpoint file is saved
    with open('data/ga_checkpoint.json', 'r') as file:
        data = json.load(file)
except FileNotFoundError:
    print("Error: 'data/ga_checkpoint.json' not found. Please ensure the path is correct.")
    exit()
except json.JSONDecodeError:
    print("Error: Could not decode JSON. The checkpoint file might be corrupted or empty.")
    exit()

# --- Extract Data from the 'history' List ---
history = data.get('history')
if not history:
    print("Error: 'history' key not found or is empty in the checkpoint file.")
    exit()

# Extract the original fitness scores and error percentages
fitness_scores = [record.get('fitness') for record in history if record.get('fitness') is not None]
ppf_errors = [record.get('ppf_error_percent') for record in history if record.get('ppf_error_percent') is not None]
keff_errors = [record.get('keff_error_percent') for record in history if record.get('keff_error_percent') is not None]

# --- Calculate the new external fitness scores ---
external_fitness_scores = []
for record in history:
    keff = record.get('keff')
    ppf = record.get('ppf')
    if keff is not None and ppf is not None:
        external_fitness_scores.append(calculate_external_fitness(ppf, keff))

if not fitness_scores:
    print("No valid data found in the history.")
    exit()

# --- Matplotlib Styling ---
plt.rcParams.update({
    'font.family': 'serif',
    'font.size': 14,
    'axes.titlesize': 20,
    'axes.labelsize': 16,
    'xtick.labelsize': 14,
    'ytick.labelsize': 14,
    'legend.fontsize': 14,
    'lines.linewidth': 2,
    'lines.markersize': 8
})

# =======================================================
# === Plot 1: Fitness Score vs. OpenMC Cycle ===
# =======================================================
print("--- Generating Fitness Plot ---")
min_fitness = min(fitness_scores)
max_fitness = max(fitness_scores)
min_index = fitness_scores.index(min_fitness)
max_index = fitness_scores.index(max_fitness)

print(f"\033[94mOriginal Fitness: Min = {min_fitness:.10f} (Cycle {min_index}), Max = {max_fitness:.10f} (Cycle {max_index})\033[0m")

plt.figure(figsize=(10, 7))
plt.plot(fitness_scores, marker='o', linestyle='-', color='darkred', label='Original Fitness Score')
plt.scatter(max_index, max_fitness, color='orange', s=100, zorder=5, label=f'Max: {max_fitness:.4f}')

plt.title('Fitness Score vs. OpenMC Cycle')
plt.xlabel('OpenMC Cycle')
plt.ylabel('Fitness Score')
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend(loc='best')
plt.tight_layout()
plt.savefig("fitness_score.png", dpi=300)
plt.show()

# =======================================================
# === Plot 2: External Fitness Score vs. OpenMC Cycle ===
# =======================================================
if external_fitness_scores:
    print("\n--- Generating External Fitness Plot ---")
    min_ext_fitness = min(external_fitness_scores)
    max_ext_fitness = max(external_fitness_scores)
    min_ext_index = external_fitness_scores.index(min_ext_fitness)
    max_ext_index = external_fitness_scores.index(max_ext_fitness)

    print(f"\033[94mExternal Fitness: Min = {min_ext_fitness:.10f} (Cycle {min_ext_index}), Max = {max_ext_fitness:.10f} (Cycle {max_ext_index})\033[0m")

    plt.figure(figsize=(10, 7))
    plt.plot(external_fitness_scores, marker='o', linestyle='-', color='darkgreen', label='External Fitness Score')
    plt.scatter(max_ext_index, max_ext_fitness, color='orange', s=100, zorder=5, label=f'Max: {max_ext_fitness:.4f}')

    plt.title('External Fitness Score vs. OpenMC Cycle')
    plt.xlabel('OpenMC Cycle')
    plt.ylabel('Fitness Score')
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.legend(loc='best')
    plt.tight_layout()
    plt.savefig("external_fitness_score.png", dpi=300)
    plt.show()
else:
    print("No External Fitness data to plot.")

# =======================================================
# === Plot 3: PPF Error Percent vs. OpenMC Cycle      ===
# =======================================================
if ppf_errors:
    print("\n--- Generating PPF Error Plot ---")
    avg_ppf = np.mean(ppf_errors)
    plt.figure(figsize=(10, 7))
    plt.plot(ppf_errors, marker='o', linestyle='-', color='royalblue', label='PPF Error %')
    plt.axhline(y=avg_ppf, color='red', linestyle='--', linewidth=1.5, label=f'Avg: {avg_ppf:.2f}%')

    plt.title('PPF Error Percent vs. OpenMC Cycle')
    plt.xlabel('OpenMC Cycle')
    plt.ylabel('PPF Error Percent (%)')
    plt.ylim(bottom=0)
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.legend(loc='best')
    plt.tight_layout()
    plt.savefig("ppf_error_percent.png", dpi=300)
    plt.show()
else:
    print("No PPF error data to plot.")

# =======================================================
# === Plot 4: Keff Error Percent vs. OpenMC Cycle     ===
# =======================================================
if keff_errors:
    print("\n--- Generating Keff Error Plot ---")
    avg_keff = np.mean(keff_errors)
    plt.figure(figsize=(10, 7))
    plt.plot(keff_errors, marker='o', linestyle='-', color='purple', label='Keff Error %')
    plt.axhline(y=avg_keff, color='red', linestyle='--', linewidth=1.5, label=f'Avg: {avg_keff:.4f}%')

    plt.title('Keff Error Percent vs. OpenMC Cycle')
    plt.xlabel('OpenMC Cycle')
    plt.ylabel('Keff Error Percent (%)')
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.legend(loc='best')
    plt.tight_layout()
    plt.savefig("keff_error_percent.png", dpi=300)
    plt.show()
else:
    print("No Keff error data to plot.")